In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import kagglehub

# ----------------------------
# CONFIG
# ----------------------------
print("Downloading ASL Alphabet dataset...")
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

DATASET_DIR = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train")
print("Dataset path:", DATASET_DIR)

IMG_SIZE = 224
BATCH_SIZE = 256
EPOCHS = 12
VAL_SPLIT = 0.15

print("TF version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

# ----------------------------
# STEP 1: Dataset
# ----------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    validation_split=VAL_SPLIT,
    subset="training",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_ds = train_ds.shuffle(1000).map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# ----------------------------
# STEP 2: Build Model
# ----------------------------
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=8e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ----------------------------
# STEP 3: Callbacks
# ----------------------------
os.makedirs("checkpoints", exist_ok=True)

callbacks = [
   ModelCheckpoint(
    filepath="checkpoints/best_stage1.keras",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False
),
    EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    )
]

# ----------------------------
# STEP 4: Train Stage 1
# ----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

loss, acc = model.evaluate(val_ds)
print("Validation accuracy after stage 1:", acc)

# ----------------------------
# STEP 5: Fine Tuning
# ----------------------------
FINE_TUNE_AT = 120

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False
for layer in base_model.layers[FINE_TUNE_AT:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

fine_callbacks = [
    ModelCheckpoint(
    filepath="checkpoints/best_finetuned.keras",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False
),
    EarlyStopping(
        monitor="val_accuracy",
        patience=2,
        restore_best_weights=True
    )
]

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=fine_callbacks
)

loss, acc = model.evaluate(val_ds)
print("Final validation accuracy:", acc)




Using Colab cache for faster access to the 'asl-alphabet' dataset.
Dataset path: /kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train
TF version: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Found 87000 files belonging to 29 classes.
Using 73950 files for training.
Found 87000 files belonging to 29 classes.
Using 13050 files for validation.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,295,133 (8.76 MB)

 Trainable params: 37,149 (145.11 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 172s 288ms/step - accuracy: 0.5376 - loss: 1.7849 - val_accuracy: 0.9521 - val_loss: 0.3043
Epoch 2/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - accuracy: 0.9259 - loss: 0.3361 - val_accuracy: 0.9769 - val_loss: 0.1616
Epoch 3/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - accuracy: 0.9549 - loss: 0.2052 - val_accuracy: 0.9816 - val_loss: 0.1138
Epoch 4/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 52ms/step - accuracy: 0.9671 - loss: 0.1495 - val_accuracy: 0.9878 - val_loss: 0.0831
Epoch 5/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - accuracy: 0.9731 - loss: 0.1231 - val_accuracy: 0.9889 - val_loss: 0.0678
Epoch 6/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 54ms/step - accuracy: 0.9769 - loss: 0.1033 - val_accuracy: 0.9915 - val_loss: 0.0582
Epoch 7/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 27s 51ms/step - accuracy: 0.9795 - loss: 0.0899 - val_accuracy: 0.9912 - val_loss: 0.0489
Epoch 8/12
289/289 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - accuracy: 0.9809 - loss: 0.0823 

ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf

In [ ]:
! git add .
! git commit -m 'model_update_v2'
! git push



In [2]:

print("Saving final macOS compatible model...")

model.save("asl_model_saved")   # <- SavedModel format (no extension)
np.save("asl_class_map.npy", {i: name for i, name in enumerate(class_names)})

print("Saved: asl_model_saved/ (SavedModel)")
print("Saved: asl_class_map.npy")


Saving final macOS compatible model...


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=asl_model_saved.